# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations
from scipy.ndimage import affine_transform


PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, \
fetch_coms, list_coms, compute_affine_transformation
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


XGBoost Version: 2.1.4


In [4]:
def sum_square_com(com):
    ss = np.sqrt(sum([s**2 for s in com]))
    return ss

def apply_affine_transformation(volume, matrix):
    """Apply an affine transformation to a 3D volume."""
    transformed_volume = affine_transform(volume, matrix, offset=0, order=1)
    return transformed_volume


In [124]:
moving_name = 'DK55'
fixed_name = 'MD589'

moving_all = list_coms(moving_name, scaling_factor=10)
fixed_all = fetch_coms(fixed_name, scaling_factor=10)

common_keys = list(moving_all.keys() & fixed_all.keys())
print(f'{moving_name} len={len(moving_all.keys())}')
print(f'{fixed_name} len={len(fixed_all.keys())}')

print(len(common_keys))

DK55 len=28
MD589 len=51
28


In [125]:
# these keys have very high errors: 'RtTg', 'AP'
midbrain_keys = {
            "3N_L",
            "3N_R",
            "4N_L",
            "4N_R",
            "IC",
            "PBG_L",
            "PBG_R",
            "SC",
            "SNC_L",
            "SNC_R",
            "SNR_L",
            "SNR_R",
        }
bad_keys = ('RtTg', 'AP', '3N_L', '3N_R')
bad_keys = ('RtTg', 'AP')
#bad_keys = ('RtTg',)
bad_keys = ()
good_keys = set(common_keys) - set(bad_keys)
print(f'#good_keys={len(good_keys)}')
print(good_keys)

#good_keys=28
{'3N_L', 'DC_R', '7N_R', 'Pn_R', '6N_R', 'LC_R', 'Pn_L', '3N_R', 'Tz_R', 'Amb_R', '5N_L', 'LRt_R', '7N_L', 'LC_L', 'AP', 'DC_L', '7n_L', 'LRt_L', '4N_L', 'Amb_L', 'PBG_L', '5N_R', '6N_L', '10N_R', 'PBG_R', 'Tz_L', '10N_L', '4N_R'}


In [126]:
moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
transformation_matrix = compute_affine_transformation(moving_src, fixed_src)
print(repr(transformation_matrix))

array([[ 8.93677412e-01, -2.03796713e-01, -2.43450053e-02,
         1.99269034e+02],
       [ 2.48660431e-01,  7.88047942e-01, -6.08070992e-02,
        -5.16511628e+01],
       [-9.52325276e-02, -5.32330406e-02,  9.34198206e-01,
         1.29293242e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])


In [127]:
# Apply affine transformation
# Apply affine transformation
origin = np.array([884, 697, 296])
print(origin)
trans_origin = apply_affine_transform(origin, transformation_matrix)
print(np.round(trans_origin))
# 100,10
#[884 697 296]
#[718. 778. 224.]

[884 697 296]
[840. 699. 285.]


In [128]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    transformed = apply_affine_transform(moving0, transformation_matrix)
    transformed = [x for x in transformed]
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 disk to Allen RMS 237.0680595008574
# MD585 to MD589 RMS 18.2658167690059

RMS 32.56014058927723


In [129]:
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
transformed_structure = apply_affine_transform(com, transformation_matrix)
print(f'{moving_name} {structure} {np.round(np.array(com))}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
print(f'{moving_name} transformed {structure} {np.round(np.array(transformed_dict[structure]))}')
#print(f'neuro res {moving_name} {structure} {np.round(com)}')


DK55 3N_L [1121.  501.  462.]
MD589 3N_L [1084.  596.  425.]
DK55 transformed 3N_L [1088.  593.  428.]


In [130]:
columns = ['structure', moving_name, fixed_name, 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(4)
df.sort_values(by=['sumsquares'], inplace=True)
HTML(df.to_html(index=False))

structure,DK55,MD589,transformed,difference,sumsquares
3N_L,"[1121.0, 501.0, 462.0]","[1084.0, 596.0, 425.0]","[1088.0, 593.0, 428.0]","[4.0, -2.0, 3.0]",5.2172
7N_R,"[1293.0, 718.0, 630.0]","[1195.0, 807.0, 552.0]","[1193.0, 798.0, 556.0]","[-2.0, -10.0, 4.0]",10.7644
LC_L,"[1258.0, 531.0, 396.0]","[1195.0, 652.0, 353.0]","[1206.0, 656.0, 351.0]","[11.0, 3.0, -1.0]",12.0008
5N_L,"[1218.0, 625.0, 340.0]","[1146.0, 710.0, 303.0]","[1153.0, 723.0, 298.0]","[6.0, 13.0, -5.0]",15.0999
3N_R,"[1117.0, 496.0, 490.0]","[1090.0, 596.0, 466.0]","[1085.0, 587.0, 454.0]","[-5.0, -9.0, -12.0]",15.7861
Tz_L,"[1201.0, 764.0, 446.0]","[1104.0, 835.0, 403.0]","[1106.0, 822.0, 391.0]","[2.0, -13.0, -12.0]",18.1047
7N_L,"[1308.0, 732.0, 366.0]","[1196.0, 842.0, 308.0]","[1210.0, 828.0, 308.0]","[14.0, -13.0, -1.0]",19.4293
5N_R,"[1220.0, 601.0, 632.0]","[1131.0, 686.0, 577.0]","[1152.0, 687.0, 572.0]","[21.0, 1.0, -6.0]",21.7953
Pn_R,"[1108.0, 742.0, 558.0]","[1019.0, 757.0, 494.0]","[1025.0, 775.0, 506.0]","[6.0, 18.0, 12.0]",21.8884
LRt_L,"[1455.0, 736.0, 368.0]","[1353.0, 860.0, 312.0]","[1340.0, 868.0, 295.0]","[-13.0, 8.0, -17.0]",22.7507
